In [9]:
import pandas as pd
import requests
import datetime
from datetime import time, timedelta

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', None)

In [2]:
API_KEY = '7d479459fdadac536d4f7cf92bda0f07'
cities = ['Margaux', 'Soussan', 'Macau, FR', 'Castelnau-de-Medoc','Lamarque, FR', 'Ludon-Medoc', 'Arsac', 'Brach, FR', 'Saint-Laurent Medoc' ]

## Current Data

In [6]:
columns_mandatory = ['observation_time','temperature','weather_code','wind_speed',
                     'wind_degree','wind_dir','pressure','precip','humidity',
                     'cloudcover','feelslike','uv_index','visibility','time','city']

df_current = pd.DataFrame(columns= columns_mandatory)
type_request = 'current'

# collect current data and create DataFrame
for city in cities:
    df_tmp = {}
    params = {
        'access_key': API_KEY,
        'query': city,
        'historical_date_start' : '2015-01-01',
        'historical_date_end' : '2016-01-01',
        'hourly' : 1,
        'interval': 3}
    api_result = requests.get(f'http://api.weatherstack.com/{type_request}', params)
    response = api_result.json()
    current = response['current']
    df_tmp['observation_time'] = response['location']['localtime']
    df_tmp['temperature'] = current['temperature']
    df_tmp['weather_code'] = current['weather_code']
    df_tmp['wind_speed'] = current['wind_speed']
    df_tmp['wind_degree'] = current['wind_degree']
    df_tmp['wind_dir'] = current['wind_dir']
    df_tmp['pressure'] = current['pressure']
    df_tmp['precip'] = current['precip']
    df_tmp['humidity'] = current['humidity']
    df_tmp['cloudcover'] = current['cloudcover']
    df_tmp['feelslike'] = current['feelslike']
    df_tmp['uv_index'] = current['uv_index']
    df_tmp['visibility'] = current['visibility']
    df_tmp['time'] = current['observation_time']
    df_tmp['city'] = city
    df_tmp = pd.DataFrame(data=df_tmp,index=[0])
    df_current = pd.concat([df_current, df_tmp])

df_current.reset_index(inplace=True, drop=True)

# Modify DataFrame to have same strutucre than historical Dataframe
for i in range(df_current.shape[0]):
    x, y = df_current['observation_time'][i].split(' ')
    df_current['observation_time'][i] = x
    time = datetime.datetime.strptime(df_current['time'][i], "%I:%M %p")
    df_current['time'][i] = time.strftime('%H:%M')

## Historical Data

In [10]:
columns_mandatory = ['observation_time','temperature','weather_code','wind_speed',
                     'wind_degree','wind_dir','pressure','precip','humidity',
                     'cloudcover','feelslike','uv_index','visibility','time','city']
columns_mandatory

['observation_time',
 'temperature',
 'weather_code',
 'wind_speed',
 'wind_degree',
 'wind_dir',
 'pressure',
 'precip',
 'humidity',
 'cloudcover',
 'feelslike',
 'uv_index',
 'visibility',
 'time',
 'city']

In [359]:
type_request = 'historical'
limit_date = datetime.date(2022, 12, 1)
response_historical = dict()
df_historical = pd.DataFrame(columns=columns_mandatory)


for city in cities:
    date_calcul = datetime.date.today()
    i=0
    while date_calcul > limit_date:
        past = date_calcul - datetime.timedelta(days=59)
        past_str = past.strftime('%Y-%m-%d')
        
        params = {
            'access_key': API_KEY,
            'query': city,
            'historical_date_start': past,
            'historical_date_end': date_calcul,
            'hourly': '1',
            'interval': '3'
        }
        
        api_result = requests.get(f'https://api.weatherstack.com/{type_request}', params)
        response_historical[i] = api_result.json()
        df_temp = pd.DataFrame.from_dict(response_historical[i]).transpose()
        df_temp = df_temp.drop(['request','location','current'], axis = 0)
        df_temp = df_temp.drop(df_temp.columns[0:29], axis = 1)
        df_temp = df_temp.transpose()
        for j in range(df_temp.shape[0]):
            df_int = df_temp.iloc[j]
            date_int = df_int['historical']['date']
            hourly_int = df_int['historical']['hourly']

            for k in range(len(hourly_int)):
                df_historical_temp = pd.DataFrame(columns=columns_mandatory)
                df_calcul = pd.DataFrame(hourly_int[k])
                df_historical_temp = pd.concat([df_historical_temp, df_calcul], axis = 0)
                df_historical_temp[columns_mandatory[0]] = date_int
                df_historical_temp['city'] = city 
                df_historical_temp = df_historical_temp[columns_mandatory]
                df_historical = pd.concat([df_historical_temp, df_historical], axis = 0)


        
        date_calcul = past
        i += 1

# Conversion of Col time from xxxx in hh:mm
df_historical['time'] = df_historical['time'].apply(lambda x: ':'.join((x[0:2],x[-2:])) if len(x) == 4 else ':'.join((x[0:1],x[-2:])))

In [ ]:
df_historical